In [1]:
using SummationByParts, LinearAlgebra
using SummationByParts.Cubature, SummationByParts.SymCubatures
using Latexify


SYSTEM: caught exception of type :MethodError while trying to print a failed Task notice; giving up


# Construct SBP Operators

Okay, we have derived some existing or new quadrature rules. How do we proceed to construct SBP operators?

## SBP Operators on Triangles (with LG or LGL facet nodes)

Consider the quadrature rule we derived for SBP diagonal-E operators.

In [2]:
qf = 11; # the facet quadrature must be qf=q+mod(q,2)+1
cub,_ = SummationByParts.Cubature.quadrature(qf, internal=false); # get LGL nodes for the facet nodes
xedge = cub.params;

In [3]:
cub_tri, vtx_tri = SummationByParts.deriveTriCubatureDiagE(q=9,
                                                    vertices=true, 
                                                    midedges=true, 
                                                    numS21=3, 
                                                    numedge=2, 
                                                    numS111=1, 
                                                    centroid=false,
                                                    delta1=1e-2, delta2=1e-2, xinit = [], xedge = xedge, verbose=false)

No solution found after a 200 LMA iterations.
No solution found after a 200 LMA iterations.
No solution found after a 200 LMA iterations.
No solution found after a 200 LMA iterations.
No solution found after a 200 LMA iterations.
No solution found after a 200 LMA iterations.
No solution found after a 200 LMA iterations.
No solution found after a 200 LMA iterations.
No solution found after a 200 LMA iterations.
----------------------------------------------------------------------------------------------
iter_pso = 9600:  iter_lma = 1813:  nperturb_pso = 30:  res norm = 2.3824494263994625e-15
----------------------------------------------------------------------------------------------


(TriSymCub{Float64}(7, 8, 33, true, true, false, 2, 3, 1, [0, 5, 3], [0.20188857323368778, 0.907619395299043, 0.509946665811286, 0.9151119481392835, 0.7344243967353571, 0.5759612484341489, 0.14306828613583494], [0.001860520647436628, 0.02283684881708678, 0.10014652323218698, 0.09204154571301754, 0.1881117789324501, 0.01681472796264025, 0.01729266187361098, 0.09672733482599313]), [-1.0 -1.0; 1.0 -1.0; -1.0 1.0])

To build SBP diagonal-E opertor using the above quadrature, we proceed as follows:

In [4]:
w, Q, E = SummationByParts.buildoperators(cub_tri, vtx_tri, 5, vertices=true)

([0.001860520647436628, 0.001860520647436628, 0.001860520647436628, 0.02283684881708678, 0.02283684881708678, 0.02283684881708678, 0.10014652323218698, 0.10014652323218698, 0.10014652323218698, 0.09204154571301754  …  0.01729266187361098, 0.01729266187361098, 0.01729266187361098, 0.01729266187361098, 0.09672733482599313, 0.09672733482599313, 0.09672733482599313, 0.09672733482599313, 0.09672733482599313, 0.09672733482599313], [-0.023809523809523774 -0.0021404716502021644 … 0.01564165372703437 0.0008432508173707295; 0.0021404716502021644 0.023809523809523774 … 0.025220881197753092 0.02189627070807618; … ; -0.01564165372703437 -0.025220881197753092 … 0.0 -0.04761221309670619; -0.0008432508173707295 -0.02189627070807618 … 0.04761221309670619 0.0;;; -0.023809523809523774 -0.0010702358251011232 … 0.015562275473809693 0.0004954374751845528; 0.0010702358251011132 0.0 … 0.0033246104896767314 -0.0033246104896766963; … ; -0.015562275473809693 -0.0033246104896767314 … 0.0 -0.09522442619341252; -0.

Now we can access all operators as follows:

In [5]:
Qx = Q[:,:,1];
Qy = Q[:,:,2];
H = diagm(w);
Ex = E[:,:,1];
Ey = E[:,:,2];
Dx = inv(H)*Qx;
Dy = inv(H)*Qy;
Sx = Qx - 0.5.*Ex;
Sy = Qy - 0.5.*Ey;

We can check if the SBP property is satisfied.

In [6]:
norm(Qx+Qx' - Ex)

1.1208447961618784e-17

In [7]:
norm(Qy+Qy' - Ey)

1.1208447961618784e-17

## SBP Operators on Tetrahedra

Consider the quadrature rule of degree 6 that we have derived previously to construct a degree $p=3$ SBP diagonal-E operator. 

In [8]:
q=6;
p=3;
qf=2*p;

In [9]:
cub_facet, vtx_facet = SummationByParts.deriveTriCubatureGamma(q=qf,
                                                       vertices=true,
                                                       midedges=false,
                                                       numS21=2,
                                                       numedge=1,
                                                       numS111=0,
                                                       centroid=false,
                                                       xinit=[],
                                                       delta1=1e-3,delta2=1e-1,verbose=false)

No solution found after a 200 LMA iterations.
----------------------------------------------------------------------------------------------
iter_pso = 1600:  iter_lma = 212:  nperturb_pso = 20:  res norm = 1.6495402437381209e-15
----------------------------------------------------------------------------------------------


(TriSymCub{Float64}(3, 4, 15, true, false, false, 1, 2, 0, [0, 3, 1], [0.23722737279318576, 0.8506802519794943, 0.3077459416259916], [0.014260718614408951, 0.20376930605390364, 0.33035897729113284, 0.05913883235361055]), [-1.0 -1.0; 1.0 -1.0; -1.0 1.0])

In [10]:
xedge = cub_facet.params
cub_tet, vtx_tet = SummationByParts.deriveTetCubatureDiagE(q=q,
                                                   vertices=true,
                                                   numS31=1,
                                                   midedges=false, 
                                                   numS22=1,
                                                   numfaceS21=2, 
                                                   numedge=1, 
                                                   numS211=0,
                                                   numfaceS111=0, 
                                                   facecentroid=false,
                                                   numS1111=0,
                                                   centroid=true,
                                                   xinit=[],
                                                   xedge=xedge,
                                                   delta1=1e-3,
                                                   delta2=1e-1,
                                                   verbose=false)

No solution found after a 200 LMA iterations.
No solution found after a 200 LMA iterations.
----------------------------------------------------------------------------------------------
iter_pso = 2400:  iter_lma = 412:  nperturb_pso = 14:  res norm = 4.352995847967053e-15
----------------------------------------------------------------------------------------------


(TetSymCub{Float64}(5, 7, 51, true, false, true, false, 1, 2, 0, 1, 1, 0, 0, [1, 2, 1, 3, 0], [0.3931484668771783, 0.7421360814368045, 0.23722737279318576, 0.8506802519794943, 0.3077459416259916], [0.0008311992138364632, 0.0670745162839674, 0.0870062463078672, 0.014487003956737916, 0.02528103240642196, 0.00465082738502153, 0.006646628516739559]), [-1.0 -1.0 -1.0; 1.0 -1.0 -1.0; -1.0 1.0 -1.0; -1.0 -1.0 1.0])

Now, we can simply construct the an SBP operator of degree $p=3$ as follows:

In [11]:
# We must provide the facet operator type, currently the availabe types are, :Omega and :DiagE
# To enforce using the facet quadrature we derived, we must also provide facetcub and facetvtx; 
# otherwise, available default facet operators saved in the code will be used
w, Q, E = SummationByParts.buildoperators(cub_tet, vtx_tet, p, faceopertype=:DiagE, facecub=cub_facet, facevtx=vtx_facet)

([0.0008311992138364632, 0.0008311992138364632, 0.0008311992138364632, 0.0008311992138364632, 0.0670745162839674, 0.0670745162839674, 0.0670745162839674, 0.0670745162839674, 0.0870062463078672, 0.0870062463078672  …  0.00465082738502153, 0.00465082738502153, 0.00465082738502153, 0.00465082738502153, 0.00465082738502153, 0.00465082738502153, 0.00465082738502153, 0.00465082738502153, 0.00465082738502153, 0.006646628516739559], [-0.007130359307204476 -0.0011963999592267882 … -5.298891963688416e-5 0.0032641516168151626; 0.0011963999592267882 0.007130359307204476 … 5.298891963686488e-5 -0.0032641516168151054; … ; 5.298891963688416e-5 -5.298891963686488e-5 … 0.0 1.633274928156334e-17; -0.0032641516168151626 0.0032641516168151054 … -1.633274928156334e-17 0.0;;; -0.007130359307204476 -0.0005981999796133801 … -0.00225634618099773 0.0032641516168151847; 0.0005981999796133801 0.0 … -0.002203357261360839 -1.8142447575624028e-17; … ; 0.00225634618099773 0.002203357261360839 … 0.029569416176805276 0

Then, we access all the remaining operators as:

In [12]:
Qx = Q[:,:,1];
Qy = Q[:,:,2];
Qz = Q[:,:,3];
H = diagm(w);
Ex = E[:,:,1];
Ey = E[:,:,2];
Ez = E[:,:,3];
Dx = inv(H)*Qx;
Dy = inv(H)*Qy;
Dz = inv(H)*Qz;
Sx = Qx - 0.5.*Ex;
Sy = Qy - 0.5.*Ey;
Sz = Qz - 0.5.*Ez;

Check if the SBP property is satisfied:

In [13]:
norm(Qx+Qx' - Ex)

0.0

In [14]:
norm(Qy+Qy' - Ey)

0.0

In [15]:
norm(Qz+Qz' - Ez)

0.0